In [7]:
import pandas as pd
dat = pd.read_csv(r"C:\Users\camda\OneDrive\Desktop\Kaggle\titanic\train.csv")
pd.set_option('display.max_rows', False)

In [14]:
dat1 = pd.DataFrame(dat)
split = int(len(dat1) * 0.75)
Y = dat1['Survived']
df = dat1.drop(['Survived'],axis=1)
df['Gender'] = (df['Sex']=='male').astype(int)
df = df.drop(['Name','Parch','Sex','Ticket','Embarked','Cabin','Age'],axis=1).fillna(0)



In [28]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import numpy as np
x = df.drop(['PassengerId'], axis=1)
x_train = x[:split]
y_train = Y[:split]
x_test = x[split:]
y_test = Y[split:]



In [29]:
scaler = preprocessing.StandardScaler().fit(x_train)
x_scaled = scaler.transform(x_train)
x_scaled_test = scaler.transform(x_test)

In [31]:
model = LogisticRegression()
model.fit(x_scaled,y_train)
y_pred = model.predict_proba(x_scaled_test)
low = 0.3
upper = 0.7
final_predictions = np.full(len(x_test), np.nan)
uncertain_indices = []
for i, prob in enumerate(y_pred[:, 1]):
    if prob <= low:
        final_predictions[i] = 0
    elif prob >= upper:
        final_predictions[i] = 1
    else:
        uncertain_indices.append(i)

In [33]:

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0,criterion='gini')
x_uncertain = x_test.iloc[uncertain_indices]
clf.fit(x_train,y_train)
y_pred2 = clf.predict(x_uncertain)
for i, index in enumerate(uncertain_indices):
    final_predictions[index] = y_pred2[i]

In [34]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy = accuracy_score(y_test, final_predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.83


In [20]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(x_train, y_train)

print("Best Parameters:", grid_search.best_params_)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 2}
